In [1]:
import csv

condition_mapping_file = "../../resources/CCSCM.csv"
procedure_mapping_file = "../../resources/CCSPROC.csv"
drug_file = "../../resources/ATC.csv"

condition_dict = {}
with open(condition_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        condition_dict[row['code']] = row['name'].lower()

procedure_dict = {}
with open(procedure_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        procedure_dict[row['code']] = row['name'].lower()

drug_dict = {}
with open(drug_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['level'] == '3':   #the prior cade was '5.0', changed to '3' to match graph_gen.ipynb
            drug_dict[row['code']] = row['name'].lower()


In [2]:
from tqdm import tqdm
import json

drug_ent = set()
drug_rel = set()

file_dir = "../../graphs/drug/ATC3"  #why in some files the level of the drug is 5 and in others 3? confusing.

for key in drug_dict.keys():
    file = f"{file_dir}/{key}.txt"
    with open(file=file, mode='r') as f:
        lines = f.readlines()
    
    for line in lines:
        parsed = line.split('\t')
        if len(parsed) == 3:
            h, r, t = line.split('\t')
            t = t[:-1]
            drug_ent.add(h)
            drug_ent.add(t)
            drug_rel.add(r)


drug_id2ent = {index: value for index, value in enumerate(drug_ent)}
drug_ent2id = {value: index for index, value in enumerate(drug_ent)}
drug_id2rel = {index: value for index, value in enumerate(drug_rel)}
drug_rel2id = {value: index for index, value in enumerate(drug_rel)}

out_file_id2ent = f"{file_dir}/id2ent.json"
out_file_ent2id = f"{file_dir}/ent2id.json"
out_file_id2rel = f"{file_dir}/id2rel.json"
out_file_rel2id = f"{file_dir}/rel2id.json"

with open(out_file_id2ent, 'w') as file:
    json.dump(drug_id2ent, file, indent=6)
with open(out_file_ent2id, 'w') as file:
    json.dump(drug_ent2id, file, indent=6)
with open(out_file_id2rel, 'w') as file:
    json.dump(drug_id2rel, file, indent=6)
with open(out_file_rel2id, 'w') as file:
    json.dump(drug_rel2id, file, indent=6)
    

In [3]:
import json

file_dir = "../../graphs/drug/ATC3"

file_id2ent = f"{file_dir}/id2ent.json"
file_ent2id = f"{file_dir}/ent2id.json"
file_id2rel = f"{file_dir}/id2rel.json"
file_rel2id = f"{file_dir}/rel2id.json"

with open(file_id2ent, 'r') as file:
    drug_id2ent = json.load(file)
with open(file_ent2id, 'r') as file:
    drug_ent2id = json.load(file)
with open(file_id2rel, 'r') as file:
    drug_id2rel = json.load(file)
with open(file_rel2id, 'r') as file:
    drug_rel2id = json.load(file)

In [8]:
from get_emb import embedding_retriever
import numpy as np
from tqdm import tqdm
import pickle
import os

# Get embedding for drug entities
drug_ent_emb = []

emb_pkl = f"{file_dir}/entity_embedding.pkl"

# Check if the file exists and load the existing embeddings
if os.path.exists(emb_pkl):
    with open(emb_pkl, "rb") as file:
        drug_ent_emb = pickle.load(file)
        # Get the length of existing embeddings
        start_idx = len(drug_ent_emb)
else:
    start_idx = 0

# Continue generating embeddings from the breakpoint
for idx in tqdm(range(start_idx, len(drug_id2ent))):
    ent = drug_id2ent[str(idx)]
    embedding = embedding_retriever(term=ent)
    embedding = np.array(embedding)
    drug_ent_emb.append(embedding)

    # Save the updated embeddings after every iteration
    stacked_embedding = np.vstack(drug_ent_emb)
    with open(emb_pkl, "wb") as file:
        pickle.dump(stacked_embedding, file)

 16%|█▌        | 3347/20859 [33:40<2:56:11,  1.66it/s]


SSLError: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/embeddings (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1129)')))

In [6]:
from get_emb import embedding_retriever
import numpy as np
from tqdm import tqdm
import pickle

## get embedding for drug entities
drug_ent_emb = []

for idx in tqdm(range(len(drug_id2ent))):
    ent = drug_id2ent[str(idx)]
    embedding = embedding_retriever(term=ent)
    embedding = np.array(embedding)
    drug_ent_emb.append(embedding)

stacked_embedding = np.vstack(drug_ent_emb)

emb_pkl = f"{file_dir}/entity_embedding.pkl"

with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

 71%|███████   | 14785/20859 [2:12:26<54:24,  1.86it/s]  


SSLError: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/embeddings (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1129)')))

In [ ]:
from get_emb import embedding_retriever
import numpy as np
from tqdm import tqdm
import pickle

## get embedding for drug relations
drug_rel_emb = []

for idx in tqdm(range(len(drug_id2rel))):
    rel = drug_id2rel[str(idx)]
    embedding = embedding_retriever(term=rel)
    embedding = np.array(embedding)
    drug_rel_emb.append(embedding)

stacked_embedding = np.vstack(drug_rel_emb)

emb_pkl = f"{file_dir}/relation_embedding.pkl"

with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

--------------------

In [ ]:
import csv

condition_mapping_file = "../../resources/CCSCM.csv"
procedure_mapping_file = "../../resources/CCSPROC.csv"
drug_file = "../../resources/ATC.csv"

condition_dict = {}
with open(condition_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        condition_dict[row['code']] = row['name'].lower()

procedure_dict = {}
with open(procedure_mapping_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        procedure_dict[row['code']] = row['name'].lower()

drug_dict = {}
with open(drug_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['level'] == '3':   #the prior cade was '5.0', changed to '3' to match graph_gen.ipynb
            drug_dict[row['code']] = row['name'].lower()


In [ ]:
from tqdm import tqdm
import json

proc_ent = set()
proc_rel = set()

file_dir = "../../graphs/procedure/CCSPROC"

for key in procedure_dict.keys():
    file = f"{file_dir}/{key}.txt"
    with open(file=file, mode='r') as f:
        lines = f.readlines()
    
    for line in lines:
        parsed = line.split('\t')
        if len(parsed) == 3:
            h, r, t = line.split('\t')
            t = t[:-1]
            proc_ent.add(h)
            proc_ent.add(t)
            proc_rel.add(r)


proc_id2ent = {index: value for index, value in enumerate(proc_ent)}
proc_ent2id = {value: index for index, value in enumerate(proc_ent)}
proc_id2rel = {index: value for index, value in enumerate(proc_rel)}
proc_rel2id = {value: index for index, value in enumerate(proc_rel)}

out_file_id2ent = f"{file_dir}/id2ent.json"
out_file_ent2id = f"{file_dir}/ent2id.json"
out_file_id2rel = f"{file_dir}/id2rel.json"
out_file_rel2id = f"{file_dir}/rel2id.json"

with open(out_file_id2ent, 'w') as file:
    json.dump(proc_id2ent, file, indent=6)
with open(out_file_ent2id, 'w') as file:
    json.dump(proc_ent2id, file, indent=6)
with open(out_file_id2rel, 'w') as file:
    json.dump(proc_id2rel, file, indent=6)
with open(out_file_rel2id, 'w') as file:
    json.dump(proc_rel2id, file, indent=6)
    

In [ ]:
import json

file_dir = "../../graphs/procedure/CCSPROC"

file_id2ent = f"{file_dir}/id2ent.json"
file_ent2id = f"{file_dir}/ent2id.json"
file_id2rel = f"{file_dir}/id2rel.json"
file_rel2id = f"{file_dir}/rel2id.json"

with open(file_id2ent, 'r') as file:
    proc_id2ent = json.load(file)
with open(file_ent2id, 'r') as file:
    proc_ent2id = json.load(file)
with open(file_id2rel, 'r') as file:
    proc_id2rel = json.load(file)
with open(file_rel2id, 'r') as file:
    proc_rel2id = json.load(file)

In [ ]:
from get_emb import embedding_retriever
import numpy as np
from tqdm import tqdm
import pickle

## get embedding for procedure entities
proc_ent_emb = []

for idx in tqdm(range(len(proc_id2ent))):
    ent = proc_id2ent[str(idx)]
    embedding = embedding_retriever(term=ent)
    embedding = np.array(embedding)
    proc_ent_emb.append(embedding)

stacked_embedding = np.vstack(proc_ent_emb)

emb_pkl = f"{file_dir}/entity_embedding.pkl"

with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)

In [ ]:
from get_emb import embedding_retriever
import numpy as np
from tqdm import tqdm
import pickle

## get embedding for procedure relations
proc_rel_emb = []

for idx in tqdm(range(len(proc_id2rel))):
    rel = proc_id2rel[str(idx)]
    embedding = embedding_retriever(term=rel)
    embedding = np.array(embedding)
    proc_rel_emb.append(embedding)

stacked_embedding = np.vstack(proc_rel_emb)

emb_pkl = f"{file_dir}/relation_embedding.pkl"

with open(emb_pkl, "wb") as file:
    pickle.dump(stacked_embedding, file)